# Clara Viz Interactive Widget
This notebook shows how to load a volume dataset using the DataDefinition class append method. The append method uses ITK to load the dataset from disk.
The rendering settings are loaded from a JSON file.
Then the Clara Viz widget is used to display an interactive view of the data.

In [ ]:
# The DataDefinition class is using ITK to load the data files, make sure ITK is available
!python3 -c "import itk" || python3 -m pip install itk

from clara.viz.core import DataDefinition
import os
from urllib.request import urlretrieve

import json

# Load the volumes (data property of S. Roettger, VIS, University of Stuttgart)
data_filename = 'bonsai_256x256x256_uint8.raw'
if not os.path.exists('data/bonsai/' + data_filename):
    print('Downloading {}'.format(data_filename))
    urlretrieve('http://cdn.klacansky.com/open-scivis-datasets/bonsai/' + data_filename, 'data/bonsai/' + data_filename)

header_filename = 'bonsai.nhdr'
if not os.path.exists('data/bonsai/' + header_filename):
    url = 'https://klacansky.com/open-scivis-datasets/metadata/bonsai.nhdr'
    print('Downloading {}'.format(header_filename))
    urlretrieve('https://klacansky.com/open-scivis-datasets/metadata/' + header_filename, 'data/bonsai/' + header_filename)

# define the render settings
with open('data/bonsai/settings.json') as f:
  settings = json.load(f)

# build the data definition
data_definition = DataDefinition()
data_definition.append('data/bonsai/' + header_filename, 'DXYZ')

print('Density volume {}x{}x{}'.format(data_definition.arrays[0].array.shape[0], data_definition.arrays[0].array.shape[1], data_definition.arrays[0].array.shape[2]))

data_definition.settings = settings

## Create a widget and select the data definition, then display the widget

In [ ]:
from clara.viz.widgets import Widget

widget = Widget()
widget.select_data_definition(data_definition)
display(widget)

## Change render settings
Interact from ipywidgets offers us a simple way to interactively control values with a callback function. In this case change the transfer functions (the mapping from the density of the source to the opacity of the visualization).
All render settings are exposed by the `settings` property of the widget. Settings are applied by calling the `set_settings` method.

In [ ]:
from ipywidgets import interact, fixed, FloatSlider

def set_range_min(index, range_min, range_max):
    if (range_max <= range_min):
        range_max = range_min + 0.01
    widget.settings['TransferFunction']['components'][index]['range']['min'] = range_min
    widget.settings['TransferFunction']['components'][index]['range']['max'] = range_max
    widget.set_settings()

for index in range(len(widget.settings['TransferFunction']['components'])):
    interact(set_range_min, index=fixed(index), range_min=FloatSlider(min=0.0, max=0.99, step=0.01, value=widget.settings['TransferFunction']['components'][index]['range']['min']),
            range_max=FloatSlider(min=0.01, max=1.0, step=0.01, value=widget.settings['TransferFunction']['components'][index]['range']['max']))